In [144]:
import pandas as pd
from sklearn_pandas import cross_val_score
from pandas import Categorical

# Load data

In [145]:
QUALITY_LABELS = ('Stub', 'Start', "C", "B", "GA", "FA")

In [146]:
input_file = "../data/labelled.enwiki.w_features.csv.gz"
revisions = pd.read_csv(input_file, index_col='revid')

In [147]:
revisions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32427 entries, 416299010 to 415876698
Data columns (total 37 columns):
anon               0 non-null float64
categories         32427 non-null int64
chars              32427 non-null int64
citation_needed    32427 non-null int64
cite_templates     32427 non-null int64
comment            29203 non-null object
content_chars      32427 non-null int64
coordinates        32427 non-null bool
external_links     32427 non-null int64
headings           32427 non-null int64
images             32427 non-null int64
infoboxes          32427 non-null int64
lead_char          32427 non-null int64
lead_paras         32427 non-null int64
main_templates     32427 non-null int64
minor              0 non-null float64
non_lead_paras     32427 non-null int64
non_ref_paras      32427 non-null int64
ns                 32427 non-null int64
other_templates    32427 non-null int64
pageid             32427 non-null int64
parentid           32427 non-null int64
ref

# Create a pipeline

In [153]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

In [154]:
from sklearn_pandas import DataFrameMapper

In [165]:
revisions['wp10'] = Categorical(revisions['wp10'])

sqrt_cols = [
             'chars',
             'content_chars',
             
             'main_templates',
             'who_templates',
             'cite_templates',
             'citation_needed',
             'infoboxes',
             'other_templates',
             
             'headings', 
             'sub_headings',
             
             'categories',
             'wikilinks',
             'images',
             'smartlists',
             
             'lead_char', 
             'lead_paras',
             
             'non_lead_paras',
             'non_ref_paras'
            ]

def gt0(x):
    return x > 0

mapper = DataFrameMapper([
    (sqrt_cols, FunctionTransformer(func=np.sqrt)),
    (['coordinates', 'infoboxes'], FunctionTransformer(func=gt0))
])


from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(multi_class='multinomial', random_state=1234)

pipe = Pipeline([
    ('mapper', mapper),
    ('clf', clf)
])


In [166]:
cvresults = cross_val_score(pipe, X=revisions.copy(), y=y, cv=5, n_jobs=-1)

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


In [160]:
cvresults

array([0.57473178, 0.56786012, 0.56311309])

In [161]:
?cross_val_score

In [162]:
??cross_val_score